In [1]:
%matplotlib notebook
#add comments
import os
import sys
import csv
import pickle
from collections import Counter
from datetime import datetime

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit

import torch
from torch import nn
from torch.utils.data import DataLoader
from network import TweetyNet
import librosa
from librosa import display
from microfaune.audio import wav2spc, create_spec, load_wav
from glob import glob

from torch.utils.data import Dataset
from CustomAudioDataset import CustomAudioDataset

from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import scipy
import IPython.display as ipd

from TweetyNetModel import TweetyNetModel  

In [2]:
train = True
fineTuning = False
#needs at least 80 for mel spectrograms ## may be able to do a little less, but must be greater than 60
n_mels=72 # The closest we can get tmeporally is 72 with an output of 432 : i think it depends on whats good
#this number should be proportional to the length of the videos. 
datasets_dir = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/Cosmos_data"
this_is_new_data = False

In [3]:
import pandas as pd
import numpy as np


#TODO Add mutliple species support for ROC curves 
def annotation_chunker_no_duplicates(kaleidoscope_df, chunk_length, include_no_bird=False, bird=None):
    """
    Function that converts a Kaleidoscope-formatted Dataframe containing 
    annotations to uniform chunks of chunk_length. If there
    are mutliple bird species in the same clip, this function creates chunks
    for the more confident bird species.

    Note: if all or part of an annotation covers the last < chunk_length
    seconds of a clip it will be ignored. If two annotations overlap in 
    the same 3 second chunk, both are represented in that chunk
    Args:
        kaleidoscope_df (Dataframe)
            - Dataframe of annotations in kaleidoscope format

        chunk_length (int)
            - duration to set all annotation chunks
    Returns:
        Dataframe of labels with chunk_length duration 
        (elements in "OFFSET" are divisible by chunk_length).
    """

    #Init list of clips to cycle through and output dataframe
    #kaleidoscope_df["FILEPATH"] =  kaleidoscope_df["FOLDER"] + kaleidoscope_df["IN FILE"] 
    kaleidoscope_df['FILEPATH'] = kaleidoscope_df.loc[:,['FOLDER','IN FILE']].sum(axis=1)
    clips = kaleidoscope_df["FILEPATH"].unique()
    df_columns = {'FOLDER': 'str', 'IN FILE' :'str', 'CLIP LENGTH' : 'float64', 'CHANNEL' : 'int64', 'OFFSET' : 'float64',
                'DURATION' : 'float64', 'SAMPLE RATE' : 'int64','MANUAL ID' : 'str'}
    output_df = pd.DataFrame({c: pd.Series(dtype=t) for c, t in df_columns.items()})
    
    # going through each clip
    for clip in clips:
        clip_df = kaleidoscope_df[kaleidoscope_df["FILEPATH"] == clip]
        path = clip_df["FOLDER"].unique()[0]
        file = clip_df["IN FILE"].unique()[0]
        birds = clip_df["MANUAL ID"].unique()
        sr = clip_df["SAMPLE RATE"].unique()[0]
        clip_len = clip_df["CLIP LENGTH"].unique()[0]

        # quick data sanitization to remove very short clips
        # do not consider any chunk that is less than chunk_length
        if clip_len < chunk_length:
            continue
        potential_annotation_count = int(clip_len)//int(chunk_length)

        # going through each species that was ID'ed in the clip
        arr_len = int(clip_len*1000)
        species_df = clip_df#[clip_df["MANUAL ID"] == bird]
        human_arr = np.zeros((arr_len))
        # looping through each annotation
        #print("========================================")
        for annotation in species_df.index:
            #print(species_df["OFFSET"][annotation])
            minval = int(round(species_df["OFFSET"][annotation] * 1000, 0))
            # Determining the end of a human label
            maxval = int(
                round(
                    (species_df["OFFSET"][annotation] +
                        species_df["DURATION"][annotation]) *
                    1000,
                    0))
            # Placing the label relative to the clip
            human_arr[minval:maxval] = 1
        # performing the chunk isolation technique on the human array

        for index in range(potential_annotation_count):
            #print("=======================")
            #print("-----------------------------------------")
            #print(index)
            chunk_start = index * (chunk_length*1000)
            chunk_end = min((index+1)*chunk_length*1000,arr_len)
            chunk = human_arr[int(chunk_start):int(chunk_end)]
            if max(chunk) >= 0.5:
                #Get row data
                row = pd.DataFrame(index = [0])
                annotation_start = chunk_start / 1000

                #Handle birdnet output edge case
                #print("-------------------------------------------")
                #print(sum(clip_df["DURATION"] == 3))
                #print(sum(clip_df["DURATION"] == 3)/clip_df.shape[0])
                #print("-------------------------------------------")
                if(sum(clip_df["DURATION"] == 3)/clip_df.shape[0] == 1):
                    #print("Processing here duration")
                    overlap = (clip_df["OFFSET"]+0.5 >= (annotation_start)) & (clip_df["OFFSET"]-0.5 <= (annotation_start))
                    annotation_df = clip_df[overlap]
                    #print(annotation_start, np.array(clip_df["OFFSET"]), overlap)
                    #print(annotation_df)
                else:
                    #print("Processing here")
                    overlap = is_overlap(clip_df["OFFSET"], clip_df["OFFSET"] + clip_df["DURATION"], annotation_start, annotation_start + chunk_length)
                    #print(overlap)
                    annotation_df = clip_df[overlap]
                    #print(annotation_df)
                
                #updating the dictionary
                if ('CONFIDENCE' in clip_df.columns):
                    annotation_df = annotation_df.sort_values(by="CONFIDENCE", ascending=False)
                    row["CONFIDENCE"] = annotation_df.iloc[0]["CONFIDENCE"]
                else:
                    #The case of manual id, or there is an annotation with no known confidence
                    row["CONFIDENCE"] = 1
                row["FOLDER"] = path
                row["IN FILE"] = file
                row["CLIP LENGTH"] = clip_len
                row["OFFSET"] = annotation_start
                row["DURATION"] = chunk_length
                row["SAMPLE RATE"] = sr
                row["MANUAL ID"] = annotation_df.iloc[0]["MANUAL ID"] 
                row["CHANNEL"] = 0
                output_df = pd.concat([output_df,row], ignore_index=True)
            elif(include_no_bird):
                #print(max(chunk))
                #Get row data
                row = pd.DataFrame(index = [0])
                annotation_start = chunk_start / 1000

                #updating the dictionary
                row["CONFIDENCE"] = 0
                row["FOLDER"] = path
                row["IN FILE"] = file
                row["CLIP LENGTH"] = clip_len
                row["OFFSET"] = annotation_start
                row["DURATION"] = chunk_length
                row["SAMPLE RATE"] = sr
                row["MANUAL ID"] = "no bird"
                row["CHANNEL"] = 0
                output_df = pd.concat([output_df,row], ignore_index=True)
    
    return output_df


def is_overlap(offset_df, end_df, chunk_start, chunk_end):
    is_both_before = (chunk_end < offset_df) & (chunk_start < offset_df)
    is_both_after = (end_df < chunk_end) & (end_df < chunk_start)
    return (~is_both_before) & (~is_both_after)
    
    
    interval = pd.Interval(left=offset_df, right=end_df)
    print(interval)


def split_save_files(dataframe_of_data):
    chunked_df = annotation_chunker_no_duplicates(dataframe_of_data, 3, include_no_bird=True)
    
    
    chunked_df.apply()
    

In [4]:
from pydub import AudioSegment
import os

data = pd.read_csv("C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing/automated_cosmos_tweety_to_birdnet.csv")
chunked_df = annotation_chunker_no_duplicates(data, 3, include_no_bird=True)
chunked_df

,FOLDER,IN FILE,CLIP LENGTH,CHANNEL,OFFSET,DURATION,SAMPLE RATE,MANUAL ID,CONFIDENCE
0,.\..\Cosmos_data\,XC100027 - Southern Nightingale-Wren - Microce...,81.528167,0,0.0,3.0,48000,Microcerculus marginatus,0.155038
1,.\..\Cosmos_data\,XC100027 - Southern Nightingale-Wren - Microce...,81.528167,0,3.0,3.0,48000,Centropus celebensis,0.020985
2,.\..\Cosmos_data\,XC100027 - Southern Nightingale-Wren - Microce...,81.528167,0,6.0,3.0,48000,Meliphaga notata,0.027436
3,.\..\Cosmos_data\,XC100027 - Southern Nightingale-Wren - Microce...,81.528167,0,9.0,3.0,48000,Microcerculus marginatus,0.331647
4,.\..\Cosmos_data\,XC100027 - Southern Nightingale-Wren - Microce...,81.528167,0,12.0,3.0,48000,Sciaphylax hemimelaena,0.017806
...,...,...,...,...,...,...,...,...,...
39712,.\..\Cosmos_data\,XC9881 - Blue-grey Tanager - Thraupis episcopu...,14.700021,0,0.0,3.0,48000,Thraupis episcopus,0.146423
39713,.\..\Cosmos_data\,XC9881 - Blue-grey Tanager - Thraupis episcopu...,14.700021,0,3.0,3.0,48000,Pyrrhocorax pyrrhocorax,0.042745
39714,.\..\Cosmos_data\,XC9881 - Blue-grey Tanager - Thraupis episcopu...,14.700021,0,6.0,3.0,48000,Thraupis sayaca,0.108350
39715,.\..\Cosmos_data\,XC9881 - Blue-grey Tanager - Thraupis episcopu...,14.700021,0,9.0,3.0,48000,no bird,0.000000


In [5]:
chunked_df.iloc[0]["IN FILE"]

'XC100027 - Southern Nightingale-Wren - Microcerculus marginatus.mp3'

In [6]:
import pydub
pydub.AudioSegment.ffmpeg = "C:/Users/Siloux/Downloads/ffmpeg-master-latest-win64-gpl/ffmpeg-master-latest-win64-gpl/bin/ffmpeg.exe"

In [7]:
from pydub import AudioSegment
AudioSegment.converter = "C:/Users/Siloux/Downloads/ffmpeg-master-latest-win64-gpl/ffmpeg-master-latest-win64-gpl/bin/ffmpeg.exe"
import os

path_to_audio = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/Cosmos_data"
data_path = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/Cosmos_data_spilt"
new_folder = data_path

from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(path_to_audio) if isfile(join(path_to_audio, f))]
chunked_df = chunked_df[chunked_df["IN FILE"].isin(onlyfiles)]


chunked_renamed_df = chunked_df.copy()
count = 0
for filename in np.unique(chunked_df["IN FILE"]):
    file_df = chunked_df[chunked_df["IN FILE"] == filename]
    file, file_type = filename.split(".")
    for index,row in file_df.iterrows():
        t1 = row["OFFSET"] * 1000
        # end time in milliseconds
        t2 = (row["OFFSET"] + 3) * 1000
        new_name = file + "_" + str(int(row["OFFSET"])) +"." + file_type
        new_path = new_folder + "/" + new_name
        
        if(this_is_new_data):
            newAudio = AudioSegment.from_mp3(path_to_audio + "/" + filename)
            newAudio = newAudio[t1:t2]
            newAudio.export(new_path, format="mp3")

        chunked_renamed_df.at[index, "IN FILE"] = new_name
        chunked_renamed_df.at[index, "FOLDER"] = new_folder
        count += 1
        print(count, "out of 6451")

        
chunked_renamed_df = chunked_renamed_df[chunked_renamed_df["IN FILE"] != "TO DELETE"]
chunked_renamed_df
    
    

1 out of 6451
2 out of 6451
3 out of 6451
4 out of 6451
5 out of 6451
6 out of 6451
7 out of 6451
8 out of 6451
9 out of 6451
10 out of 6451
11 out of 6451
12 out of 6451
13 out of 6451
14 out of 6451
15 out of 6451
16 out of 6451
17 out of 6451
18 out of 6451
19 out of 6451
20 out of 6451
21 out of 6451
22 out of 6451
23 out of 6451
24 out of 6451
25 out of 6451
26 out of 6451
27 out of 6451
28 out of 6451
29 out of 6451
30 out of 6451
31 out of 6451
32 out of 6451
33 out of 6451
34 out of 6451
35 out of 6451
36 out of 6451
37 out of 6451
38 out of 6451
39 out of 6451
40 out of 6451
41 out of 6451
42 out of 6451
43 out of 6451
44 out of 6451
45 out of 6451
46 out of 6451
47 out of 6451
48 out of 6451
49 out of 6451
50 out of 6451
51 out of 6451
52 out of 6451
53 out of 6451
54 out of 6451
55 out of 6451
56 out of 6451
57 out of 6451
58 out of 6451
59 out of 6451
60 out of 6451
61 out of 6451
62 out of 6451
63 out of 6451
64 out of 6451
65 out of 6451
66 out of 6451
67 out of 6451
68 o

592 out of 6451
593 out of 6451
594 out of 6451
595 out of 6451
596 out of 6451
597 out of 6451
598 out of 6451
599 out of 6451
600 out of 6451
601 out of 6451
602 out of 6451
603 out of 6451
604 out of 6451
605 out of 6451
606 out of 6451
607 out of 6451
608 out of 6451
609 out of 6451
610 out of 6451
611 out of 6451
612 out of 6451
613 out of 6451
614 out of 6451
615 out of 6451
616 out of 6451
617 out of 6451
618 out of 6451
619 out of 6451
620 out of 6451
621 out of 6451
622 out of 6451
623 out of 6451
624 out of 6451
625 out of 6451
626 out of 6451
627 out of 6451
628 out of 6451
629 out of 6451
630 out of 6451
631 out of 6451
632 out of 6451
633 out of 6451
634 out of 6451
635 out of 6451
636 out of 6451
637 out of 6451
638 out of 6451
639 out of 6451
640 out of 6451
641 out of 6451
642 out of 6451
643 out of 6451
644 out of 6451
645 out of 6451
646 out of 6451
647 out of 6451
648 out of 6451
649 out of 6451
650 out of 6451
651 out of 6451
652 out of 6451
653 out of 6451
654 out 

1166 out of 6451
1167 out of 6451
1168 out of 6451
1169 out of 6451
1170 out of 6451
1171 out of 6451
1172 out of 6451
1173 out of 6451
1174 out of 6451
1175 out of 6451
1176 out of 6451
1177 out of 6451
1178 out of 6451
1179 out of 6451
1180 out of 6451
1181 out of 6451
1182 out of 6451
1183 out of 6451
1184 out of 6451
1185 out of 6451
1186 out of 6451
1187 out of 6451
1188 out of 6451
1189 out of 6451
1190 out of 6451
1191 out of 6451
1192 out of 6451
1193 out of 6451
1194 out of 6451
1195 out of 6451
1196 out of 6451
1197 out of 6451
1198 out of 6451
1199 out of 6451
1200 out of 6451
1201 out of 6451
1202 out of 6451
1203 out of 6451
1204 out of 6451
1205 out of 6451
1206 out of 6451
1207 out of 6451
1208 out of 6451
1209 out of 6451
1210 out of 6451
1211 out of 6451
1212 out of 6451
1213 out of 6451
1214 out of 6451
1215 out of 6451
1216 out of 6451
1217 out of 6451
1218 out of 6451
1219 out of 6451
1220 out of 6451
1221 out of 6451
1222 out of 6451
1223 out of 6451
1224 out of 64

1667 out of 6451
1668 out of 6451
1669 out of 6451
1670 out of 6451
1671 out of 6451
1672 out of 6451
1673 out of 6451
1674 out of 6451
1675 out of 6451
1676 out of 6451
1677 out of 6451
1678 out of 6451
1679 out of 6451
1680 out of 6451
1681 out of 6451
1682 out of 6451
1683 out of 6451
1684 out of 6451
1685 out of 6451
1686 out of 6451
1687 out of 6451
1688 out of 6451
1689 out of 6451
1690 out of 6451
1691 out of 6451
1692 out of 6451
1693 out of 6451
1694 out of 6451
1695 out of 6451
1696 out of 6451
1697 out of 6451
1698 out of 6451
1699 out of 6451
1700 out of 6451
1701 out of 6451
1702 out of 6451
1703 out of 6451
1704 out of 6451
1705 out of 6451
1706 out of 6451
1707 out of 6451
1708 out of 6451
1709 out of 6451
1710 out of 6451
1711 out of 6451
1712 out of 6451
1713 out of 6451
1714 out of 6451
1715 out of 6451
1716 out of 6451
1717 out of 6451
1718 out of 6451
1719 out of 6451
1720 out of 6451
1721 out of 6451
1722 out of 6451
1723 out of 6451
1724 out of 6451
1725 out of 64

2233 out of 6451
2234 out of 6451
2235 out of 6451
2236 out of 6451
2237 out of 6451
2238 out of 6451
2239 out of 6451
2240 out of 6451
2241 out of 6451
2242 out of 6451
2243 out of 6451
2244 out of 6451
2245 out of 6451
2246 out of 6451
2247 out of 6451
2248 out of 6451
2249 out of 6451
2250 out of 6451
2251 out of 6451
2252 out of 6451
2253 out of 6451
2254 out of 6451
2255 out of 6451
2256 out of 6451
2257 out of 6451
2258 out of 6451
2259 out of 6451
2260 out of 6451
2261 out of 6451
2262 out of 6451
2263 out of 6451
2264 out of 6451
2265 out of 6451
2266 out of 6451
2267 out of 6451
2268 out of 6451
2269 out of 6451
2270 out of 6451
2271 out of 6451
2272 out of 6451
2273 out of 6451
2274 out of 6451
2275 out of 6451
2276 out of 6451
2277 out of 6451
2278 out of 6451
2279 out of 6451
2280 out of 6451
2281 out of 6451
2282 out of 6451
2283 out of 6451
2284 out of 6451
2285 out of 6451
2286 out of 6451
2287 out of 6451
2288 out of 6451
2289 out of 6451
2290 out of 6451
2291 out of 64

...

20349 out of 6451
20350 out of 6451
20351 out of 6451
20352 out of 6451
20353 out of 6451
20354 out of 6451
20355 out of 6451
20356 out of 6451
20357 out of 6451
20358 out of 6451
20359 out of 6451
20360 out of 6451
20361 out of 6451
20362 out of 6451
20363 out of 6451
20364 out of 6451
20365 out of 6451
20366 out of 6451
20367 out of 6451
20368 out of 6451
20369 out of 6451
20370 out of 6451
20371 out of 6451
20372 out of 6451
20373 out of 6451
20374 out of 6451
20375 out of 6451
20376 out of 6451
20377 out of 6451
20378 out of 6451
20379 out of 6451
20380 out of 6451
20381 out of 6451
20382 out of 6451
20383 out of 6451
20384 out of 6451
20385 out of 6451
20386 out of 6451
20387 out of 6451
20388 out of 6451
20389 out of 6451
20390 out of 6451
20391 out of 6451
20392 out of 6451
20393 out of 6451
20394 out of 6451
20395 out of 6451
20396 out of 6451
20397 out of 6451
20398 out of 6451
20399 out of 6451
20400 out of 6451
20401 out of 6451
20402 out of 6451
20403 out of 6451
20404 out 

20895 out of 6451
20896 out of 6451
20897 out of 6451
20898 out of 6451
20899 out of 6451
20900 out of 6451
20901 out of 6451
20902 out of 6451
20903 out of 6451
20904 out of 6451
20905 out of 6451
20906 out of 6451
20907 out of 6451
20908 out of 6451
20909 out of 6451
20910 out of 6451
20911 out of 6451
20912 out of 6451
20913 out of 6451
20914 out of 6451
20915 out of 6451
20916 out of 6451
20917 out of 6451
20918 out of 6451
20919 out of 6451
20920 out of 6451
20921 out of 6451
20922 out of 6451
20923 out of 6451
20924 out of 6451
20925 out of 6451
20926 out of 6451
20927 out of 6451
20928 out of 6451
20929 out of 6451
20930 out of 6451
20931 out of 6451
20932 out of 6451
20933 out of 6451
20934 out of 6451
20935 out of 6451
20936 out of 6451
20937 out of 6451
20938 out of 6451
20939 out of 6451
20940 out of 6451
20941 out of 6451
20942 out of 6451
20943 out of 6451
20944 out of 6451
20945 out of 6451
20946 out of 6451
20947 out of 6451
20948 out of 6451
20949 out of 6451
20950 out 

21452 out of 6451
21453 out of 6451
21454 out of 6451
21455 out of 6451
21456 out of 6451
21457 out of 6451
21458 out of 6451
21459 out of 6451
21460 out of 6451
21461 out of 6451
21462 out of 6451
21463 out of 6451
21464 out of 6451
21465 out of 6451
21466 out of 6451
21467 out of 6451
21468 out of 6451
21469 out of 6451
21470 out of 6451
21471 out of 6451
21472 out of 6451
21473 out of 6451
21474 out of 6451
21475 out of 6451
21476 out of 6451
21477 out of 6451
21478 out of 6451
21479 out of 6451
21480 out of 6451
21481 out of 6451
21482 out of 6451
21483 out of 6451
21484 out of 6451
21485 out of 6451
21486 out of 6451
21487 out of 6451
21488 out of 6451
21489 out of 6451
21490 out of 6451
21491 out of 6451
21492 out of 6451
21493 out of 6451
21494 out of 6451
21495 out of 6451
21496 out of 6451
21497 out of 6451
21498 out of 6451
21499 out of 6451
21500 out of 6451
21501 out of 6451
21502 out of 6451
21503 out of 6451
21504 out of 6451
21505 out of 6451
21506 out of 6451
21507 out 

21943 out of 6451
21944 out of 6451
21945 out of 6451
21946 out of 6451
21947 out of 6451
21948 out of 6451
21949 out of 6451
21950 out of 6451
21951 out of 6451
21952 out of 6451
21953 out of 6451
21954 out of 6451
21955 out of 6451
21956 out of 6451
21957 out of 6451
21958 out of 6451
21959 out of 6451
21960 out of 6451
21961 out of 6451
21962 out of 6451
21963 out of 6451
21964 out of 6451
21965 out of 6451
21966 out of 6451
21967 out of 6451
21968 out of 6451
21969 out of 6451
21970 out of 6451
21971 out of 6451
21972 out of 6451
21973 out of 6451
21974 out of 6451
21975 out of 6451
21976 out of 6451
21977 out of 6451
21978 out of 6451
21979 out of 6451
21980 out of 6451
21981 out of 6451
21982 out of 6451
21983 out of 6451
21984 out of 6451
21985 out of 6451
21986 out of 6451
21987 out of 6451
21988 out of 6451
21989 out of 6451
21990 out of 6451
21991 out of 6451
21992 out of 6451
21993 out of 6451
21994 out of 6451
21995 out of 6451
21996 out of 6451
21997 out of 6451
21998 out 

ValueError: too many values to unpack (expected 2)

In [ ]:
x = chunked_renamed_df.groupby(["IN FILE", "OFFSET"]).count().sort_values(by="MANUAL ID", ascending=False)
x.shape[0]

In [ ]:
def show_mel_spectrogram(spec, uid):
    fig, ax = plt.subplots()
    M_db_bird = librosa.power_to_db(spec, ref=np.max)
    img = librosa.display.specshow(M_db_bird, y_axis='mel', x_axis='time', ax=ax)
    ax.set(title=uid)
    plt.show()

In [ ]:
def load_dataset(data_path, use_dump=True, csv="labels.csv", kalediscope=False):
    mel_dump_file = os.path.join(data_path, "mel_dataset.pkl")
    if os.path.exists(mel_dump_file) and use_dump:
        with open(mel_dump_file, "rb") as f:
            dataset = pickle.load(f)
    
    elif kalediscope:
        dataset = compute_feature_kaledoscope_labels(data_path, csv)
        with open(mel_dump_file, "wb") as f:
            pickle.dump(dataset, f)
    else:
        dataset = compute_feature(data_path, csv)
        with open(mel_dump_file, "wb") as f:
            pickle.dump(dataset, f)
    inds = [i for i, x in enumerate(dataset["X"]) if x.shape[1] == 431]
    X = np.array([dataset["X"][i].transpose() for i in inds])
    Y = np.array([int(dataset["Y"][i]) for i in inds])
    uids = [dataset["uids"][i] for i in inds]
    return X, Y, uids

def compute_feature(data_path, csv = "labels.csv"):
    print(f"Compute features for dataset {os.path.basename(data_path)}")
    labels_file = os.path.join(data_path, csv)
    print(labels_file)
    if os.path.exists(labels_file):
        with open(labels_file, "r") as f:
            reader = csv.reader(f, delimiter=',')
            labels = {}
            next(reader)  # pass fields names
            for name, _, y in reader:
                labels[name] = y
    else:
        print("Warning: no label file detected.")
        wav_files = glob(os.path.join(data_path, "wav/*.wav"))
        labels = {os.path.basename(f)[:-4]: None for f in wav_files}
    i = 1
    X = []
    Y = []
    uids = []
    for file_id, y in labels.items():
        print(f"{i:04d}/{len(labels)}: {file_id:20s}", end="\r")
        spc = wav2spc(os.path.join(data_path, "wav", f"{file_id}.wav"), n_mels=n_mels)
        X.append(spc)
        Y.append(y)
        uids.append(file_id)
        i += 1
    return {"uids": uids, "X": X, "Y": Y}

def compute_feature_df(data_path, df):
    
    i = 1
    X = []
    Y = []
    uids = []
    for file_id, y in labels.items():
        print(f"{i:04d}/{len(labels)}: {file_id:20s}", end="\r")
        spc = wav2spc(os.path.join(data_path, "wav", f"{file_id}.wav"), n_mels=n_mels)
        X.append(spc)
        Y.append(y)
        uids.append(file_id)
        i += 1
    return {"uids": uids, "X": X, "Y": Y}

def split_dataset(X, Y, test_size=0.2, random_state=0):
    split_generator = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    ind_train, ind_test = next(split_generator.split(X, Y))
    X_train, X_test = X[ind_train, :, :], X[ind_test, :, :]
    Y_train, Y_test = Y[ind_train], Y[ind_test]
    return ind_train, ind_test

In [ ]:
species = list(np.unique(chunked_renamed_df["MANUAL ID"]))
chunked_renamed_df["y"] = chunked_renamed_df["MANUAL ID"].apply(lambda x: species.index(x))
chunked_renamed_df

In [ ]:
import os
from pydub import AudioSegment

# You dont need the number of files in the folder, just iterate over them directly using:
if (this_is_new_data):
    for index, row in chunked_renamed_df.iterrows():
        file = row["IN FILE"]
        folder = row["FOLDER"]
        #spliting the file into the name and the extension
        name, ext = os.path.splitext(folder + '/' + file)
        if ext == ".mp3":
            #os.remove(folder + '/' + file) 

            mp3_sound = AudioSegment.from_mp3(folder + '/' + file)
            mp3_sound.export("{0}.wav".format(name), format="wav")

In [ ]:
import os
from pydub import AudioSegment

if(this_is_new_data):
    # You dont need the number of files in the folder, just iterate over them directly using:
    for index, row in chunked_renamed_df.iterrows():
        file = row["IN FILE"]
        folder = row["FOLDER"]
        #spliting the file into the name and the extension
        name, ext = os.path.splitext(file)
        if ext == ".mp3":
            print(folder)
            try:
                os.remove(folder + "/" + file) 
            except Exception as e:
                print(e)
            

In [ ]:
chunked_renamed_df["IN FILE"][201]

In [ ]:
chunked_renamed_df["IN FILE"] = chunked_renamed_df["IN FILE"].apply(lambda x: x.replace(".mp3", ".wav"))

In [ ]:
import os
from pydub import AudioSegment

# You dont need the number of files in the folder, just iterate over them directly using:
if(this_is_new_data):
    for index, row in chunked_renamed_df.iterrows():
        file = row["IN FILE"]
        folder = row["FOLDER"]
        sound = AudioSegment.from_wav(folder + "/" + file)
        sound = sound.set_channels(1)
        sound.export(folder + "/" + file, format="wav")


In [ ]:
data_path = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/Cosmos_data_spilt"
#chunked_renamed_df = pd.read_csv("formated_tweetynet_ground_truth.csv")
i = 1
X = []
Y = []
uids = []
for index, row in chunked_renamed_df.iterrows():
    try:
        print(f"{i:04d}/{chunked_renamed_df.shape[0]}", end="\r")
        print(os.path.join(data_path, row["IN FILE"]))
        spc = wav2spc(os.path.join(data_path, row["IN FILE"]), n_mels=n_mels)
        X.append(spc)
        Y.append(row["y"])
        uids.append(row["IN FILE"])
        i += 1
    except:
        continue
dataset = {"uids": uids, "X": X, "Y": Y}

In [ ]:
dataset

In [ ]:


inds = [i for i, x in enumerate(dataset["X"]) if x.shape[1] == 130]
X = np.array([dataset["X"][i].transpose() for i in inds])
Y = np.array([int(dataset["Y"][i]) for i in inds])
uids = [dataset["uids"][i] for i in inds]
X, Y, uids


In [ ]:
dataset["X"][0].shape[1]


In [ ]:
dataset

In [ ]:
#X0, Y0, uids0 = load_dataset(datasets_dir, csv="automated_cosmos_tweety_to_file.csv")#os.path.join(datasets_dir, "ff1010bird_wav"))
#print(X0)
#uids0, X0, Y0 =  uids, X,Y#load_dataset(os.path.join(datasets_dir, "warblrb10k_public_wav"))

#print(X0.shape)
#print(Y0.shape)
#print(len(uids0))

#X = np.concatenate([X0]).astype(np.float32)/255
#Y = np.concatenate([Y0])
#uids = np.concatenate([uids0])
#print(X.shape, Y.shape, uids.shape)
#del X0, Y0, uids0, X1, Y1, uids1 

In [ ]:
Counter(Y)

Actual work

In [ ]:
print(type(uids))
uids = np.array(uids)

In [ ]:
ind_train_val, ind_test = split_dataset(X, Y)
ind_train, ind_val = split_dataset(X[ind_train_val, :, :, np.newaxis], Y[ind_train_val], test_size=0.1)
X_train, X_test, X_val = X[ind_train, :, :, np.newaxis], X[ind_test, :, :, np.newaxis], X[ind_val, :, :, np.newaxis]
Y_train, Y_test, Y_val = Y[ind_train], Y[ind_test], Y[ind_val]
uids_train, uids_test, uids_val = uids[ind_train], uids[ind_test], uids[ind_val]
#del X, Y

print("Training set: ", Counter(Y_train))
print("Test set: ", Counter(Y_test))
print("Validation set: ", Counter(Y_val))
#very imbalanced dataset

In [ ]:
len(Counter(Y_train))
X_train[0].shape

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
tweetynet = TweetyNetModel(len(Counter(Y_train)), (1, n_mels, 431), device, binary=True)

In [ ]:
train_dataset = CustomAudioDataset(X_train, Y_train, uids_train)
test_dataset = CustomAudioDataset(X_test, Y_test, uids_test)
val_dataset = CustomAudioDataset(X_val, Y_val, uids_val)

In [ ]:
#train_dataset = train_dataset.type(torch.LongTensor)
#test_dataset = test_dataset.type(torch.LongTensor)
#val_dataset = val_dataset.type(torch.LongTensor)

In [ ]:
train_dataset[:10]

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload




history, test_out, start_time, end_time = tweetynet.train_pipeline(train_dataset, val_dataset, test_dataset, 
                                                                   lr=.01, batch_size=128,epochs=10, save_me=True,
                                                                   fine_tuning=False, finetune_path=None)

In [ ]:
len(train_dataset)

In [ ]:
end_time - start_time

In [ ]:
test_out


In [ ]:
chunked_renamed_df.to_csv("formated_tweetynet_ground_truth.csv")

# Visualization

In [ ]:
#should categorize this data

In [ ]:
a, sr = librosa.load(os.path.join(datasets_dir, folder_name, "wav", uid+".wav"))
sr

In [ ]:
#-100,-2
idx_here = -100
x = X[idx_here]
y = Y[idx_here]
uid = uids[idx_here]
folder_name = "ff1010bird_wav" if "-" not in uid else "warblrb10k_public_wav"
sr = 44100
x.shape

In [ ]:
show_mel_spectrogram(x.transpose(), uid)
y, uid

In [ ]:
ipd.Audio(os.path.join(datasets_dir, folder_name, "wav", uid+".wav"))

In [ ]:
plt.figure(figsize=(14, 5))
librosa.display.waveplot(a, sr=sr)